In [ ]:
# Cell 1: Imports
import pandas as pd
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import average_precision_score, precision_recall_curve

In [ ]:
# Cell 2: Load data
X_train = pd.read_csv("../data/raw/X_train_G3tdtEn.csv")
y_train = pd.read_csv("../data/raw/Y_train_2_XPXJDyy.csv")
X_test = pd.read_csv("../data/raw/X_test_8skS2ey.csv")

/var/folders/3x/jrcv617s1gd_94yvvqdgzdrr0000gn/T/ipykernel_3103/377774935.py:2: DtypeWarning: Columns (21,22,23,24,69,70,71,72,93,94,95,96,97,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv("../data/raw/X_train_G3tdtEn.csv")
/var/folders/3x/jrcv617s1gd_94yvvqdgzdrr0000gn/T/ipykernel_3103/377774935.py:4: DtypeWarning: Columns (20,21,22,23,24,68,69,70,71,72,92,93,94,95,96,97,106,107,108,109,110,111,112,113,114,115,116,117,118) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv("../data/raw/X_test_8skS2ey.csv")


In [ ]:
# Cell 3: Format label
y_train = y_train["fraud_flag"]

In [ ]:
# Cell 4: Preprocessing pipeline
num_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64'] and col != 'ID']
cat_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, num_cols),
    ("cat", categorical_transformer, cat_cols)
])

In [ ]:
# Cell 5: Train/test split
X_train_part, X_val_part, y_train_part, y_val_part = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

X_train_part[cat_cols] = X_train_part[cat_cols].astype(str)
X_val_part[cat_cols] = X_val_part[cat_cols].astype(str)

In [ ]:
# Cell 6: Define stacking model (optimized for speed)
base_learners = [
    ("rf", RandomForestClassifier(n_estimators=30, random_state=42, n_jobs=-1, class_weight="balanced")),
    ("lgbm", LGBMClassifier(n_estimators=30, random_state=42, n_jobs=-1, class_weight="balanced")),
    ("xgb", XGBClassifier(n_estimators=30, random_state=42, n_jobs=-1, eval_metric="logloss", scale_pos_weight=5))
]

stack_model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", StackingClassifier(
        estimators=base_learners,
        final_estimator=LogisticRegression(max_iter=500, class_weight="balanced"),
        n_jobs=-1
    ))
])

In [ ]:
# Cell 7: Fit stacking model
stack_model.fit(X_train_part, y_train_part)

In [ ]:
# Cell 8: Evaluate on validation
y_val_proba = stack_model.predict_proba(X_val_part)[:, 1]
pr_auc = average_precision_score(y_val_part, y_val_proba)
print(f"Stacked Model Validation PR-AUC: {pr_auc:.4f}")

In [ ]:
# Cell 9: Retrain on full training set and predict
X_train[cat_cols] = X_train[cat_cols].astype(str)
X_test[cat_cols] = X_test[cat_cols].astype(str)
stack_model.fit(X_train, y_train)

# Save model
os.makedirs("../output", exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
joblib.dump(stack_model, f"../output/stacking_model_{timestamp}.joblib")

# Predict on test
y_test_pred = stack_model.predict_proba(X_test)[:, 1]

# Format submission
submission = pd.DataFrame({
    "index": range(len(X_test)),
    "ID": X_test["ID"],
    "fraud_flag": y_test_pred
})

# Validation du format
expected_cols = ["index", "ID", "fraud_flag"]
assert submission.shape[1] == 3, "Le fichier de soumission doit avoir 3 colonnes : index, ID, fraud_flag"
assert submission.columns.tolist() == expected_cols, f"Les colonnes doivent être exactement {expected_cols}"
assert submission["fraud_flag"].between(0, 1).all(), "Toutes les probabilités doivent être entre 0 et 1"
assert submission["ID"].is_unique, "Chaque ID doit être unique"
assert (submission["index"] == range(len(submission))).all(), "La colonne index doit être une séquence de 0 à N-1"

submission.to_csv("../output/submission_stacking.csv", index=False)
print("✅ Submission saved to ../output/submission_stacking.csv")

In [ ]:
# Cell 10: Plot Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_train, stack_model.predict_proba(X_train)[:, 1])
plt.figure(figsize=(8, 5))
plt.plot(recall, precision, label=f"PR-AUC = {pr_auc:.2f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (Stacked Model)")
plt.legend()
plt.grid()
plt.show()

[LightGBM] [Info] Number of positive: 1055, number of negative: 73177
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.358267 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6752
[LightGBM] [Info] Number of data points in the train set: 74232, number of used features: 1411
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
